<a href="https://colab.research.google.com/github/CogitoNTNU/course-on-large-language-models/blob/main/Course_on_Large_Language_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Course on Supervised Machine Learning

In [6]:
%pip install numpy matplotlib scipy pandas seaborn scikit-learn statsmodels


[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## Load data

In [8]:
training_data: pd.DataFrame = pd.read_csv('data/train.csv')
test_data: pd.DataFrame = pd.read_csv('data/test.csv')
training_data.describe()

,TicketClass,Age,amountOfSiblingAndSpousesOnBoard,amountOfParentsAndChildrenOnBoard,Fare,Survived
count,712.000000,572.000000,712.000000,712.000000,712.000000,712.000000
mean,2.330056,29.498846,0.553371,0.379213,32.586276,0.376404
std,0.824584,14.500059,1.176404,0.791669,51.969529,0.484824
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,21.000000,0.000000,0.000000,7.925000,0.000000
50%,3.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,3.000000,38.000000,1.000000,0.000000,30.500000,1.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000


## TASK 1: separate the features from the target


In [14]:
# TODO: Name the target column "Survived" and store it in y
target_column = 'Survived'
y = training_data[target_column]
X = training_data.drop(columns=[target_column])

X.head()
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Survived, dtype: int64

## Exploratory data analysis

## Preprocessing of data
When we are working with data, we need to preprocess it before we can use it to train a model. This preprocessing can include:
* **Handling Missing Values**:
  * **Imputation**: Replace missing values with a specific value like the mean, median, mode, or a constant value. This method is chosen based on the nature of the data and the type of variable (continuous or categorical).
  * **Deletion**: Remove rows or columns with missing values, typically used when the proportion of missing data is minimal or if the missing data is not random.

* **Encoding categorical variables**:
* **Feature Scaling**:
  * Normalization (Min-Max Scaling): Scales the features to a fixed range, usually 0 to 1, or -1 to 1. 
* **Handling Outliers**:
  * **Trimming**: Remove extreme values.
  * **Capping**: Apply thresholds to limit the range of variable values.


## Train models

## Make predictions